<a href="https://colab.research.google.com/github/chrismlawr/BA780-Team4A/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting our Google Drive to this notebook to access the dataset

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/Shared\ drives/BA775\ -\ Team\ 4A

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/Shared drives/BA775 - Team 4A


----

Importing packages and loading the dataset to our notebook

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import seaborn as sns

In [ ]:
raw = pd.read_csv('listings.csv')

In [ ]:
raw.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20201005145859,2020-10-07,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, New York, United States",A New Yorker since 2000! My passion is creatin...,within a day,70%,25%,f,https://a0.muscache.com/im/pictures/user/50fc5...,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,6.0,6.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"New York, United States",Midtown,Manhattan,40.75362,-73.98377,Entire apartment,Entire home/apt,2,NaN,1 bath,NaN,1.0,"[""Hot water"", ""Stove"", ""Extra pillows and blan...",$175.00,3,1125,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,30,60,90,365,2020-10-07,48,1,0,2009-11-21,2019-11-04,94.0,9.0,9.0,10.0,10.0,10.0,9.0,NaN,f,2,2,0,0,0.36
1,3831,https://www.airbnb.com/rooms/3831,20201005145859,2020-10-09,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,LisaRoxanne,2008-12-07,"New York, New York, United States",Laid-back Native New Yorker (formerly bi-coast...,within an hour,98%,96%,f,https://a0.muscache.com/im/users/4869/profile_...,https://a0.muscache.com/im/users/4869/profile_...,Clinton Hill,1.0,1.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"Brooklyn, New York, United States",Clinton Hill,Brooklyn,40.68514,-73.95976,Entire guest suite,Entire home/apt,3,NaN,1 bath,1.0,3.0,"[""Hot water"", ""Stove"", ""Free parking on premis...",$76.00,1,730,1.0,2.0,1125.0,1125.0,1.0,1125.0,NaN,t,2,2,2,2,2020-10-09,354,70,3,2014-09-30,2020-09-20,89.0,8.0,9.0,9.0,9.0,9.0,9.0,NaN,f,1,1,0,0,4.82
2,5121,https://www.airbnb.com/rooms/5121,20201005145859,2020-10-09,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,Garon,2009-02-03,"New York, New York, United States","I am an artist(painter, filmmaker) and curato...",NaN,NaN,100%,f,https://a0.muscache.com/im/pictures/user/72a61...,https://a0.muscache.com/im/pictures/user/72a61...,Bedford-Stuyvesant,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,t,NaN,Bedford-Stuyvesant,Brooklyn,40.68688,-73.95596,Private room in apartment,Private room,2,NaN,NaN,1.0,1.0,"[""Wifi"", ""Air conditioning"", ""Heating"", ""Kitch...",$60.00,29,730,29.0,29.0,730.0,730.0,29.0,730.0,NaN,t,2,2,2,2,2020-10-09,50,1,0,2009-05-28,2019-12-

In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44666 entries, 0 to 44665
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            44666 non-null  int64  
 1   listing_url                                   44666 non-null  object 
 2   scrape_id                                     44666 non-null  int64  
 3   last_scraped                                  44666 non-null  object 
 4   name                                          44650 non-null  object 
 5   description                                   43385 non-null  object 
 6   neighborhood_overview                         28207 non-null  object 
 7   picture_url                                   44666 non-null  object 
 8   host_id                                       44666 non-null  int64  
 9   host_url                                      44666 non-null 

In [ ]:
round(raw.describe(),2)

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,44666.00,4.466600e+04,4.466600e+04,44649.00,44649.00,44666.00,44666.00,44666.00,0.0,40407.00,44153.00,44666.00,4.466600e+04,44660.00,44660.00,4.466000e+04,4.466000e+04,44660.00,4.466000e+04,0.0,44666.00,44666.00,44666.00,44666.00,44666.00,44666.00,44666.00,33102.00,33070.00,33083.00,33055.00,33075.00,33050.00,33051.00,0.0,44666.00,44666.00,44666.00,44666.00,34149.00
mean,23227109.25,2.020101e+13,8.531840e+07,15.82,15.82,40.73,-73.95,2.76,NaN,1.29,1.51,9.30,4.956784e+04,9.42,13.31,1.155654e+06,2.309702e+06,12.96,2.213198e+06,NaN,8.14,18.57,29.67,105.42,22.46,4.15,0.19,93.85,9.60,9.27,9.72,9.73,9.59,9.39,NaN,6.67,3.62,2.85,0.09,0.86
std,13982124.33,2.850000e+00,1.001533e+08,117.94,117.94,0.06,0.05,1.80,NaN,0.68,1.09,24.23,1.016199e+07,29.03,57.99,4.976976e+07,7.036601e+07,56.60,6.881096e+07,NaN,12.06,24.99,37.93,139.52,46.66,9.89,0.91,9.42,0.92,1.14,0.81,0.83,0.80,0.99,NaN,24.85,19.91,13.94,0.94,1.27
min,2595.00,2.020101e+13,2.438000e+03,0.00,0.00,40.51,-74.24,0.00,NaN,1.00,0.00,1.00,1.000000e+00,1.00,1.00,1.000000e+00,1.000000e+00,1.00,1.000000e+00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,2.00,2.00,2.00,2.00,2.00,2.00,NaN,1.00,0.00,0.00,0.00,0.01
25%,10555067.75,2.020101e+13,9.354841e+06,1.00,1.00,40.69,-73.98,2.00,NaN,1.00,1.00,2.00,2.900000e+01,2.00,2.00,3.000000e+01,3.100000e+01,2.00,3.100000e+01,NaN,0.00,0.00,0.00,0.00,1.00,0.00,0.00,92.00,9.00,9.00,10.00,10.00,9.00,9.00,NaN,1.00,0.00,0.00,0.00,0.11
50%,22266056.00,2.020101e+13,3.785493e+07,1.00,1.00,40.72,-73.96,2.00,NaN,1.00,1.00,3.00,3.650000e+02,3.00,3.00,1.125000e+03,1.125000e+03,3.00,1.125000e+03,NaN,0.00,0.00,0.00,1.00,4.00,0.00,0.00,97.00,10.00,10.00,10.00,10.00,10.00,10.00,NaN,1.00,1.00,1.00,0.00,0.35
75%,36239497.50,2.020101e+13,1.373589e+08,2.00,2.00,40.76,-73.93,4.00,NaN,1.00,2.00,7.00,1.125000e+03,7.00,7.00,1.125000e+03,1.125000e+03,7.00,1.125000e+03,NaN,20.00,47.00,73.00,189.00,20.00,4.00,0.00,100.00,10.00,10.00,10.00,10.00,10.00,10.00,NaN,2.00,1.00,1.00,0.00,1.13
max,45730924.00,2.020101e+13,3.705121e+08,2114.00,2114.00,40.91,-73.71,16.00,NaN,21.00,40.00,1250.00,2.147484e+09,1250.00,1250.00,2.147484e+09,2.147484e+09,1250.00,2.147484e+09,NaN,30.00,60.00,90.00,365.00,746.00,473.00,44.00,100.00,10.00,10.00,10.00,10.00,10.00,10.00,NaN,239.00,239.00,134.00,28.00,41.37


## Checking for Duplicates

Checking for duplicates in this dataset is very nuanced. There may be a case where a data point could be a duplicate or could be a listing identical in description but actually different listings (e.g. a host that has two listings that are very similar that are both rentable). This means that there is some manual checking and personal judgement that may be required to determine whether or not the data row is a duplicate.

In [ ]:
raw.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [ ]:
dup = raw.duplicated(subset=['id'], keep=False)
raw[dup]

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month


It seems there are no duplicate entries in our dataset based on the ID of the listing. With that said, there may be duplicate listings under the same conditions.

In [ ]:
dup = raw.duplicated(subset=['name','latitude','longitude','host_id'], keep=False)
raw[dup].sort_values('name').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 25597 to 257
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            53 non-null     int64  
 1   listing_url                                   53 non-null     object 
 2   scrape_id                                     53 non-null     int64  
 3   last_scraped                                  53 non-null     object 
 4   name                                          53 non-null     object 
 5   description                                   53 non-null     object 
 6   neighborhood_overview                         1 non-null      object 
 7   picture_url                                   53 non-null     object 
 8   host_id                                       53 non-null     int64  
 9   host_url                                      53 non-null     

As seen above, there do seem to be a number of duplicates based on the listing name, location, and host. The question is now whether or not we should deal with these duplicates.

The issue is these duplicate rows may be...


*   errors (e.g. host creates a new listing and forgets to delete the old one)
*   intentional (e.g. a hotel lists 10 identical but different rooms)
*   or both (e.g. [host creates duplicate of single listing to increase listing exposure](https://www.reddit.com/r/AirBnB/comments/3w9uo7/duplicate_listings/))

Ultimately, since there are no data input / scrape errors (shown by unique listing IDs), we will not tamper with these 'duplicates'.

##Dropping columns

Since we're utilizing a dataset scraped from AirBnB's website, our new dataset contains a lot more variables than our first dataset from Kaggle, where it seems like the author in Kaggle had already cleaned and excluded irrelevant columns.

We will now do the same with the variables in this dataset that we won't be using for this project. Most of the columns we will drop relates to the host, web scraping, and listing information that are unlikely to effect price (e.g. host url, scrape id, host location, etc.)

In [ ]:
cols_to_drop = ['host_url','calendar_updated','listing_url','scrape_id','last_scraped','neighborhood_overview','picture_url','host_name','host_since','host_response_time','host_location','host_about','host_thumbnail_url','host_picture_url','host_neighbourhood','host_listings_count','host_verifications','neighbourhood','bathrooms','calendar_last_scraped','first_review','last_review','license','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms']

In [ ]:
clean_droppedcols = raw.drop(columns=cols_to_drop)

In [ ]:
clean_droppedcols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44666 entries, 0 to 44665
Data columns (total 47 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            44666 non-null  int64  
 1   name                          44650 non-null  object 
 2   description                   43385 non-null  object 
 3   host_id                       44666 non-null  int64  
 4   host_response_rate            21587 non-null  object 
 5   host_acceptance_rate          27801 non-null  object 
 6   host_is_superhost             44649 non-null  object 
 7   host_total_listings_count     44649 non-null  float64
 8   host_has_profile_pic          44649 non-null  object 
 9   host_identity_verified        44649 non-null  object 
 10  neighbourhood_cleansed        44666 non-null  object 
 11  neighbourhood_group_cleansed  44666 non-null  object 
 12  latitude                      44666 non-null  float64
 13  l

##Adding an *Amenities_Count* column

With amenities of a listing listed, it might be interesting to see if the amount of amenities could affect the listing's price

In [ ]:
clean_droppedcols['amenities'][0]

'["Hot water", "Stove", "Extra pillows and blankets", "Smoke alarm", "Laptop-friendly workspace", "Coffee maker", "Bed linens", "Dishes and silverware", "Oven", "Wifi", "Air conditioning", "Paid parking off premises", "Baking sheet", "Fire extinguisher", "Free street parking", "Essentials", "Heating", "Kitchen", "Carbon monoxide alarm", "Cooking basics", "Refrigerator", "Hair dryer", "Hangers", "Keypad", "Bathtub", "Ethernet connection", "TV", "Iron", "Cleaning before checkout", "Long term stays allowed", "Luggage dropoff allowed"]'

In [ ]:
len(clean_droppedcols['amenities'][0].strip("][").split('", "'))

31

In [ ]:
amenities_count = []
for i in range(len(clean_droppedcols['amenities'])):
    count = len(clean_droppedcols['amenities'][i].strip("][").split('", "'))
    amenities_count.append(count)

In [ ]:
clean_droppedcols['amenities_count'] = amenities_count

In [ ]:
clean_droppedcols['amenities_count'][0]

31

## Fixing *price* variable (and other object vars)

When taking a closer look at the price of the listings, it seems that the data type is a string with dollar signs afixed to the front and commas seperating thousands. We need to convert this into a float so we can run analysis on it later on.

In [ ]:
data = clean_droppedcols

In [ ]:
type(data['price'][0])

str

In [ ]:
data['price'] = [float(data['price'][i].strip('$').strip(',').replace(',','')) for i in range(len(data['price']))]

In [ ]:
# note the change in type
type(data['price'][0])

numpy.float64

In [ ]:
raw['price'].describe()

count       44666
unique       1163
top       $100.00
freq         1633
Name: price, dtype: object

In [ ]:
# data['price'].describe()

In [ ]:
# plt.figure(figsize=(30, 6))
# sns.boxplot('price',data=data, whis=1.5)

In [ ]:
data['bathrooms_text'].unique()

array(['1 bath', nan, '1.5 baths', '1 shared bath', '1 private bath',
       'Shared half-bath', '2 baths', '3.5 baths', '1.5 shared baths',
       '2.5 baths', '3 baths', 'Half-bath', '2 shared baths',
       '0 shared baths', '0 baths', '5 baths', 'Private half-bath',
       '4.5 baths', '5.5 baths', '2.5 shared baths', '15.5 baths',
       '6 baths', '4 baths', '3 shared baths', '8 shared baths',
       '4 shared baths', '3.5 shared baths', '6 shared baths',
       '6.5 shared baths', '6.5 baths', '5 shared baths',
       '4.5 shared baths', '7.5 baths', '5.5 shared baths', '7 baths',
       '8 baths'], dtype=object)

## Dealing with null variables

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44666 entries, 0 to 44665
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            44666 non-null  int64  
 1   name                          44650 non-null  object 
 2   description                   43385 non-null  object 
 3   host_id                       44666 non-null  int64  
 4   host_response_rate            21587 non-null  object 
 5   host_acceptance_rate          27801 non-null  object 
 6   host_is_superhost             44649 non-null  object 
 7   host_total_listings_count     44649 non-null  float64
 8   host_has_profile_pic          44649 non-null  object 
 9   host_identity_verified        44649 non-null  object 
 10  neighbourhood_cleansed        44666 non-null  object 
 11  neighbourhood_group_cleansed  44666 non-null  object 
 12  latitude                      44666 non-null  float64
 13  l

Taking a look at the information on our dataset, the first existance of null values is seen in the names and description columns.

In [ ]:
data[data['description'].isna()][['price','number_of_reviews']].head(7)

,price,number_of_reviews
2201,390.0,1
2232,171.0,30
2568,220.0,2
2930,200.0,28
2932,300.0,0
3235,40.0,2
3802,30.0,0


In [ ]:
data[data['name'].isna()][['price','number_of_reviews']].head(7)

,price,number_of_reviews
2270,400.0,0
2930,200.0,28
4613,225.0,1
4767,215.0,5
5009,150.0,0
5246,70.0,0
5277,40.0,0


Considering the fact that this project does not plan to use *description* or *name* in our exploration, it may be worth to leave the columns as is for the moment (perhaps even dropping it entirely later on).

### Null values related to host

The next few columns with null values pertain to the host, specifically their interaction with customers.

In [ ]:
data[data['host_response_rate'].isna()][['price','reviews_per_month']].head()

,price,reviews_per_month
2,60.0,0.36
5,75.0,0.87
8,85.0,0.89
15,86.0,NaN
17,55.0,1.27


In [ ]:
data[data['host_acceptance_rate'].isna()][['price','reviews_per_month']].head()

,price,reviews_per_month
3,175.0,0.01
5,75.0,0.87
15,86.0,NaN
26,120.0,0.09
44,250.0,0.10


The queries above show that despite null values, the listings do seem to be active.

Information on the [AirBnB page](https://www.airbnb.com/help/article/430/what-are-response-rate-and-response-time-and-how-are-they-calculated) show that these values are based on the last 30 days, so if the host has not interacted with customers for the past month, they will have null values. For this project, we will fill the null values with the mean values.

### host_response_rate

In [ ]:
data['host_response_rate'].fillna('-1%',inplace=True)

In [ ]:
data['host_response_rate'] = [int(data['host_response_rate'][i].strip('%')) for i in range(len(data['host_response_rate']))]

In [ ]:
# # filter = data['host_response_rate'] >= 0
# mean_host_resp_rate = data[filter]['host_response_rate'].mean()
# data['host_response_rate'].replace({-1:mean_host_resp_rate})


### host_acceptance_rate

In [ ]:
data['host_acceptance_rate'].fillna('-1%',inplace=True)

In [ ]:
data['host_acceptance_rate'] = [int(data['host_acceptance_rate'][i].strip('%')) for i in range(len(data['host_acceptance_rate']))]

In [ ]:
# filter = data['host_acceptance_rate'] >= 0
# mean_host_acc_rate = data[filter]['host_acceptance_rate'].mean()
# data['host_acceptance_rate'].replace({-1:mean_host_acc_rate})

## Code for Export

In [ ]:
data.to_csv('cleaned_listings.csv', index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44666 entries, 0 to 44665
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            44666 non-null  int64  
 1   name                          44650 non-null  object 
 2   description                   43385 non-null  object 
 3   host_id                       44666 non-null  int64  
 4   host_response_rate            44666 non-null  int64  
 5   host_acceptance_rate          44666 non-null  int64  
 6   host_is_superhost             44649 non-null  object 
 7   host_total_listings_count     44649 non-null  float64
 8   host_has_profile_pic          44649 non-null  object 
 9   host_identity_verified        44649 non-null  object 
 10  neighbourhood_cleansed        44666 non-null  object 
 11  neighbourhood_group_cleansed  44666 non-null  object 
 12  latitude                      44666 non-null  float64
 13  l